In [ ]:
"""
Hand Tracing Module
By: Shubham Tiwari
Website: https://github.com/ShubhGurukul

"""

import cv2
import mediapipe as mp
import time
import math
import numpy as np


class handDetector():
    def __init__(self, mode=False, maxHands=2, detectionCon=0.5, trackCon=0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.trackCon = trackCon

        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHands,
                                        self.detectionCon, self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils
        self.tipIds = [4, 8, 12, 16, 20]

    def findHands(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        # print(results.multi_hand_landmarks)

        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms,
                                               self.mpHands.HAND_CONNECTIONS)

        return img

    def findPosition(self, img, handNo=0, draw=True):
        xList = []
        yList = []
        bbox = []
        self.lmList = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(myHand.landmark):
                # print(id, lm)
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                xList.append(cx)
                yList.append(cy)
                # print(id, cx, cy)
                self.lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)

            xmin, xmax = min(xList), max(xList)
            ymin, ymax = min(yList), max(yList)
            bbox = xmin, ymin, xmax, ymax

            if draw:
                cv2.rectangle(img, (xmin - 20, ymin - 20), (xmax + 20, ymax + 20),
                              (0, 255, 0), 2)

        return self.lmList, bbox

    def fingersUp(self):
        fingers = []
        # Thumb
        if self.lmList[self.tipIds[0]][1] > self.lmList[self.tipIds[0] - 1][1]:
            fingers.append(1)
        else:
            fingers.append(0)

        # Fingers
        for id in range(1, 5):

            if self.lmList[self.tipIds[id]][2] < self.lmList[self.tipIds[id] - 2][2]:
                fingers.append(1)
            else:
                fingers.append(0)

        # totalFingers = fingers.count(1)

        return fingers

    def findDistance(self, p1, p2, img, draw=True,r=15, t=3):
        x1, y1 = self.lmList[p1][1:]
        x2, y2 = self.lmList[p2][1:]
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

        if draw:
            cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), t)
            cv2.circle(img, (x1, y1), r, (255, 0, 255), cv2.FILLED)
            cv2.circle(img, (x2, y2), r, (255, 0, 255), cv2.FILLED)
            cv2.circle(img, (cx, cy), r, (0, 0, 255), cv2.FILLED)
        length = math.hypot(x2 - x1, y2 - y1)

        return length, img, [x1, y1, x2, y2, cx, cy]


def main():
    pTime = 0
    cTime = 0
    cap = cv2.VideoCapture(1)
    detector = handDetector()
    while True:
        success, img = cap.read()
        img = detector.findHands(img)
        lmList, bbox = detector.findPosition(img)
        if len(lmList) != 0:
            print(lmList[4])

        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime

        cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3,
                    (255, 0, 255), 3)

        cv2.imshow("Image", img)
        cv2.waitKey(1)


if __name__ == "__main__":
    main()

[4, 428, 383]
[4, 410, 314]
[4, 403, 270]
[4, 413, 254]
[4, 417, 239]
[4, 423, 229]
[4, 419, 222]
[4, 412, 222]
[4, 399, 227]
[4, 383, 236]
[4, 377, 242]
[4, 367, 257]
[4, 363, 265]
[4, 357, 270]
[4, 358, 272]
[4, 358, 275]
[4, 359, 275]
[4, 359, 275]
[4, 359, 275]
[4, 357, 275]
[4, 353, 276]
[4, 353, 275]
[4, 351, 274]
[4, 350, 274]
[4, 350, 274]
[4, 349, 274]
[4, 349, 274]
[4, 349, 275]
[4, 349, 276]
[4, 350, 277]
[4, 350, 276]
[4, 350, 276]
[4, 350, 274]
[4, 348, 277]
[4, 350, 276]
[4, 348, 276]
[4, 349, 276]
[4, 349, 276]
[4, 349, 276]
[4, 349, 276]
[4, 349, 271]
[4, 348, 270]
[4, 344, 265]
[4, 342, 262]
[4, 342, 259]
[4, 338, 256]
[4, 327, 252]
[4, 330, 250]
[4, 327, 248]
[4, 328, 247]
[4, 327, 249]
[4, 325, 247]
[4, 326, 247]
[4, 325, 247]
[4, 324, 249]
[4, 327, 247]
[4, 325, 248]
[4, 325, 249]
[4, 328, 247]
[4, 332, 256]
[4, 329, 259]
[4, 330, 256]
[4, 330, 259]
[4, 327, 261]
[4, 331, 259]
[4, 327, 261]
[4, 326, 263]
[4, 329, 265]
[4, 324, 265]
[4, 323, 264]
[4, 320, 264]
[4, 31

In [ ]:
import cv2
import mediapipe as mp
import time
 
cap = cv2.VideoCapture(0)
 
mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils
 
pTime = 0
cTime = 0
 
while True:
    success, img = cap.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    # print(results.multi_hand_landmarks)
 
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            for id, lm in enumerate(handLms.landmark):
                # print(id, lm)
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                print(id, cx, cy)
                # if id == 4:
                cv2.circle(img, (cx, cy), 15, (255, 0, 255), cv2.FILLED)
 
            mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)
 
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
 
    cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3,
                (255, 0, 255), 3)
 
    cv2.imshow("Image", img)
    cv2.waitKey(1)

In [1]:
# %%writefile HandTrackingModule.py
"""
Hand Tracing Module
By: Shubham Tiwari
Website: https://github.com/ShubhGurukul

"""
import cv2
import mediapipe as mp
import time
class handDetector():
    def __init__(self, mode=False, maxHands=2, detectionCon=0.5, trackCon=0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.trackCon = trackCon
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHands,
                                        self.detectionCon, self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils
    def findHands(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        # print(results.multi_hand_landmarks)
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms,
                                               self.mpHands.HAND_CONNECTIONS)
        return img
    def findPosition(self, img, handNo=0, draw=True):
        lmList = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(myHand.landmark):
                # print(id, lm)
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                # print(id, cx, cy)
                lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 15, (255, 0, 255), cv2.FILLED)
        return lmList
def main():
    pTime = 0
    cTime = 0
    cap = cv2.VideoCapture(1)
    detector = handDetector()
    while True:
        success, img = cap.read()
        img = detector.findHands(img)
        lmList = detector.findPosition(img)
        if len(lmList) != 0:
            print(lmList[4])
        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime
        cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3,
                    (255, 0, 255), 3)
        cv2.imshow("Image", img)
        cv2.waitKey(1)
if __name__ == "__main__":
    main()


Writing HandTrackingModule.py


In [7]:
!pip install pycaw

  Created wheel for pycaw: filename=pycaw-20181226-py3-none-any.whl size=6502 sha256=f4b18e77fd33f55a6fdc14e494b02aa9d5669110bb57d59c29c9f4cfe2f3b823
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\64\9f\fb\2beb4fd0539384ea6f234caa8a9485cf12e70681115151b1dd
Successfully built pycaw


In [8]:
import cv2
import time
import numpy as np
import HandTrackingModule as htm
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
################################
wCam, hCam = 640, 480
################################
cap = cv2.VideoCapture(1)
cap.set(3, wCam)
cap.set(4, hCam)
pTime = 0
detector = htm.handDetector(detectionCon=0.7)
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
# volume.GetMute()
# volume.GetMasterVolumeLevel()
volRange = volume.GetVolumeRange()
minVol = volRange[0]
maxVol = volRange[1]
vol = 0
volBar = 400
volPer = 0
while True:
    success, img = cap.read()
    img = detector.findHands(img)
    lmList = detector.findPosition(img, draw=False)
    if len(lmList) != 0:
        # print(lmList[4], lmList[8])
        x1, y1 = lmList[4][1], lmList[4][2]
        x2, y2 = lmList[8][1], lmList[8][2]
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        cv2.circle(img, (x1, y1), 15, (255, 0, 255), cv2.FILLED)
        cv2.circle(img, (x2, y2), 15, (255, 0, 255), cv2.FILLED)
        cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), 3)
        cv2.circle(img, (cx, cy), 15, (255, 0, 255), cv2.FILLED)
        length = math.hypot(x2 - x1, y2 - y1)
        # print(length)
        # Hand range 50 - 300
        # Volume Range -65 - 0
        vol = np.interp(length, [50, 300], [minVol, maxVol])
        volBar = np.interp(length, [50, 300], [400, 150])
        volPer = np.interp(length, [50, 300], [0, 100])
        print(int(length), vol)
        volume.SetMasterVolumeLevel(vol, None)
        if length <50:
            cv2.circle(img, (cx, cy), 15, (0, 255, 0), cv2.FILLED)
    cv2.rectangle(img, (50, 150), (85, 400), (255, 0, 0), 3)
    cv2.rectangle(img, (50, int(volBar)), (85, 400), (255, 0, 0), cv2.FILLED)
    cv2.putText(img, f'{int(volPer)} %', (40, 450), cv2.FONT_HERSHEY_COMPLEX,
                1, (255, 0, 0), 3)
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    cv2.putText(img, f'FPS: {int(fps)}', (40, 50), cv2.FONT_HERSHEY_COMPLEX,
                1, (255, 0, 0), 3)
    cv2.imshow("Img", img)
    cv2.imshow("Video",img)     # and showing on screen
    if cv2.waitKey(1) & 0xff == ord('q'):   
        break

159 -36.61673925902629
162 -35.87645912939374
169 -34.08610696851045
181 -30.840323800093202
178 -31.598500484459812
190 -28.6516527465415
190 -28.64616477048323
189 -28.804874967326327
187 -29.292428166659796
187 -29.267414375743954
182 -30.56910095127057
181 -30.810908347284638
179 -31.38021358744266
179 -31.38021358744266
180 -31.139098291063178
176 -32.114798116712755
177 -31.91243541637479
177 -31.91243541637479
173 -33.08594012920317
165 -35.18441153132663
160 -36.48620340126958
155 -37.637573707413864
146 -40.16719435971174
99 -52.44913908590277
65 -61.22094493246187
45 -65.25
34 -65.25
27 -65.25
29 -65.25
23 -65.25
26 -65.25
20 -65.25
21 -65.25
21 -65.25
22 -65.25
21 -65.25
23 -65.25
22 -65.25
21 -65.25
22 -65.25
30 -65.25
37 -65.25
43 -65.25
46 -65.25
71 -59.61524351242436
88 -55.30828497045077
100 -51.96614764224573
117 -47.628335389157634
122 -46.28199055219079
130 -44.2917901382622
147 -39.82573158590277
151 -38.79835919863581
176 -32.14799648985972
178 -31.778992164829447


132 -43.6331472152432
133 -43.39911747534169
135 -43.04663811776244
135 -43.049536726448544
134 -43.28317547235329
134 -43.28317547235329
134 -43.16567716605313
136 -42.582076096167064
135 -42.81455671687333
136 -42.582076096167064
136 -42.699561730225845
134 -43.16470774278375
135 -42.815516574141874
127 -45.1160284926593
134 -43.16567716605313
130 -44.216757592623324
129 -44.56530827175087
129 -44.44537217749986
129 -44.55723197483644
128 -44.66743832236384
128 -44.66743832236384
129 -44.43028432360259
130 -44.192782039574084
131 -44.061216595795926
129 -44.53805851257958
128 -44.7759749135042
130 -44.29980338292144
130 -44.192782039574084
130 -44.192782039574084
130 -44.29980338292144
130 -44.192782039574084
130 -44.192782039574084
130 -44.32084714122496
132 -43.60957500980999
118 -47.24977434864603
104 -50.97091944466481
66 -60.99902679037968
58 -63.09464982317079
59 -62.89215508255616
38 -65.25
28 -65.25
17 -65.25
12 -65.25
12 -65.25
10 -65.25
19 -65.25
13 -65.25
19 -65.25
21 -65.

30 -65.25
38 -65.25
94 -53.67730924533226
98 -52.546824351160105
110 -49.546138050689606
63 -61.72495873308304
61 -62.225705209870014
56 -63.674681541928734
110 -49.358406816486415
117 -47.75853878741228
89 -55.05780711722751
90 -54.77377792759747
36 -65.25
30 -65.25
16 -65.25
17 -65.25
13 -65.25
11 -65.25
19 -65.25
18 -65.25
20 -65.25
18 -65.25
11 -65.25
8 -65.25
11 -65.25
11 -65.25
5 -65.25
9 -65.25
16 -65.25
41 -65.25
47 -65.25
63 -61.65114571509499
84 -56.337195056186474
89 -55.04755116122175
91 -54.52606601758977
94 -53.766
95 -53.504999999999995
98 -52.68873082020338
108 -50.10112709699198
112 -49.04936308385747
93 -53.781276032387005
83 -56.63542776789719
99 -52.375459772640134
114 -48.52768984442087
126 -45.412964302023205
122 -46.37147411796154
115 -48.193222922405
128 -44.882825478505815
127 -45.087301329160255
82 -56.88368161891498
52 -64.70543222459794
37 -65.25
7 -65.25
21 -65.25
3 -65.25
16 -65.25
51 -64.96599047547963
90 -54.78536292008741
92 -54.084600602096195
101 -51.

168 -34.432584655578346
179 -31.423789647626165
182 -30.55126442511802
163 -35.72818767071338
178 -31.726306974859725
185 -29.856543785563765
188 -29.158475298379273
186 -29.62157577529856
187 -29.272971893046588
186 -29.507657281085592
188 -29.158475298379273
185 -29.86005940342205
183 -30.319992976240613
181 -30.90352104849982
171 -33.619422273206894
172 -33.39813771122627
158 -36.81741834215232
147 -39.76734740509031
136 -42.64602433107915
113 -48.70094290353154
102 -51.67672061897708
93 -53.80212233682273
84 -56.36978659474559
72 -59.34557692779862
39 -65.25
25 -65.25
20 -65.25
9 -65.25
7 -65.25
8 -65.25
14 -65.25
43 -65.25
43 -65.25
58 -62.98083406317433
78 -57.74718216399513
88 -55.170922781053285
108 -50.05285371935777
111 -49.12748463022086
129 -44.5663179448196
136 -42.62310432506774
145 -40.45140017120845
153 -38.146695926735994
157 -37.08348027792739
150 -39.02491980912069
161 -36.26603305658625
162 -35.792241590504915
164 -35.463387423840345
168 -34.45122322116613
169 -34.0

87 -55.352770755492074
81 -57.15255653749135
68 -60.338344842415
55 -63.945
49 -65.25
36 -65.25
35 -65.25
43 -65.25
57 -63.259061166270236
110 -49.50707430635087
145 -40.265574067694935
185 -29.933946408663857
239 -15.910626385577487
247 -13.79497073095773
255 -11.687195690317914
256 -11.43049603144943
256 -11.23061716401439
261 -10.12751945249461
266 -8.859283514641056
260 -10.43548284265185
259 -10.68840464831495
244 -14.417350806341787
179 -31.370777440916406
110 -49.53547904101304
70 -59.9018139752855
31 -65.25
27 -65.25
37 -65.25
70 -59.96115115390282
169 -34.12618371704795
212 -22.724157577235047
263 -9.620787118663046
273 -6.985839435635221
291 -2.236521431109921
268 -8.186635767494373
290 -2.4746709799423243
290 -2.5650133689851344
290 -2.524548236252656
276 -6.21815046074358
279 -5.401994471727818
294 -1.538484525121568
287 -3.2440053226925727
289 -2.806003781757589
275 -6.30893249159309
298 -0.4003535181320643
294 -1.538484525121568
279 -5.345947816450938
58 -63.1260427046864

# Finger Counter 

In [2]:
'''
Finger Counter Module
By: Shubham Tiwari
Website: https://github.com/ShubhGurukul
'''
import cv2
import time
import os
import HandTrackingModule as htm
wCam, hCam = 640, 480
cap = cv2.VideoCapture(1)
cap.set(3, wCam)
cap.set(4, hCam)
folderPath = "FingerImages"
myList = os.listdir(folderPath)
print(myList)
overlayList = []
for imPath in myList:
    image = cv2.imread(f'{folderPath}/{imPath}')
    # print(f'{folderPath}/{imPath}')
    overlayList.append(image)
print(len(overlayList))
pTime = 0
detector = htm.handDetector(detectionCon=0.75)
tipIds = [4, 8, 12, 16, 20]
while True:
    success, img = cap.read()
    img = detector.findHands(img)
    lmList = detector.findPosition(img, draw=False)
    # print(lmList)
    if len(lmList) != 0:
        fingers = []
        # Thumb
        if lmList[tipIds[0]][1]< lmList[tipIds[0] - 1][1]:
            fingers.append(1)
        else:
            fingers.append(0)
        # 4 Fingers
        for id in range(1, 5):
            if lmList[tipIds[id]][2] < lmList[tipIds[id] - 2][2]:
                fingers.append(1)
            else:
                fingers.append(0)
        # print(fingers)
        totalFingers = fingers.count(1)
        print(totalFingers)
        h, w, c = overlayList[totalFingers - 1].shape
        img[0:h, 0:w] = overlayList[totalFingers - 1]
        cv2.rectangle(img, (20, 225), (170, 425), (0, 255, 0), cv2.FILLED)
        cv2.putText(img, str(totalFingers), (45, 375), cv2.FONT_HERSHEY_PLAIN,
                    10, (255, 0, 0), 25)
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    cv2.putText(img, f'FPS: {int(fps)}', (400, 70), cv2.FONT_HERSHEY_PLAIN,
                3, (255, 0, 0), 3)
    cv2.imshow("Image", img)
#     cv2.waitKey(1)
    if cv2.waitKey(1) & 0xff == ord('q'):   
        break

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'FingerImages'

# AI Personal Trainer

## PoseModule

In [4]:
'''
Pose Module
By: Shubham Tiwari
Website: https://github.com/ShubhGurukul

'''
# %%writefile PoseModule.py
import cv2
import mediapipe as mp
import time
import math


class poseDetector():

    def __init__(self, mode=False, upBody=False, smooth=True,
                 detectionCon=0.5, trackCon=0.5):

        self.mode = mode
        self.upBody = upBody
        self.smooth = smooth
        self.detectionCon = detectionCon
        self.trackCon = trackCon

        self.mpDraw = mp.solutions.drawing_utils
        self.mpPose = mp.solutions.pose
        self.pose = self.mpPose.Pose(self.mode, self.upBody, self.smooth,
                                     self.detectionCon, self.trackCon)

    def findPose(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.pose.process(imgRGB)
        if self.results.pose_landmarks:
            if draw:
                self.mpDraw.draw_landmarks(img, self.results.pose_landmarks,
                                           self.mpPose.POSE_CONNECTIONS)
        return img

    def findPosition(self, img, draw=True):
        self.lmList = []
        if self.results.pose_landmarks:
            for id, lm in enumerate(self.results.pose_landmarks.landmark):
                h, w, c = img.shape
                # print(id, lm)
                cx, cy = int(lm.x * w), int(lm.y * h)
                self.lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 5, (255, 0, 0), cv2.FILLED)
        return self.lmList

    def findAngle(self, img, p1, p2, p3, draw=True):

        # Get the landmarks
        x1, y1 = self.lmList[p1][1:]
        x2, y2 = self.lmList[p2][1:]
        x3, y3 = self.lmList[p3][1:]

        # Calculate the Angle
        angle = math.degrees(math.atan2(y3 - y2, x3 - x2) -
                             math.atan2(y1 - y2, x1 - x2))
        if angle &amp;lt; 0:
            angle += 360

        # print(angle)

        # Draw
        if draw:
            cv2.line(img, (x1, y1), (x2, y2), (255, 255, 255), 3)
            cv2.line(img, (x3, y3), (x2, y2), (255, 255, 255), 3)
            cv2.circle(img, (x1, y1), 10, (0, 0, 255), cv2.FILLED)
            cv2.circle(img, (x1, y1), 15, (0, 0, 255), 2)
            cv2.circle(img, (x2, y2), 10, (0, 0, 255), cv2.FILLED)
            cv2.circle(img, (x2, y2), 15, (0, 0, 255), 2)
            cv2.circle(img, (x3, y3), 10, (0, 0, 255), cv2.FILLED)
            cv2.circle(img, (x3, y3), 15, (0, 0, 255), 2)
            cv2.putText(img, str(int(angle)), (x2 - 50, y2 + 50),
                        cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)
        return angle

def main():
    cap = cv2.VideoCapture('PoseVideos/1.mp4')
    pTime = 0
    detector = poseDetector()
    while True:
        success, img = cap.read()
        img = detector.findPose(img)
        lmList = detector.findPosition(img, draw=False)
        if len(lmList) != 0:
            print(lmList[14])
            cv2.circle(img, (lmList[14][1], lmList[14][2]), 15, (0, 0, 255), cv2.FILLED)

        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime

        cv2.putText(img, str(int(fps)), (70, 50), cv2.FONT_HERSHEY_PLAIN, 3,
                    (255, 0, 0), 3)

        cv2.imshow("Image", img)
        cv2.waitKey(1)


if __name__ == "__main__":
    main()

Writing PoseModule.py


## AI Trainer

In [ ]:
import cv2
import numpy as np
import time
import PoseModule as pm
# cap = cv2.VideoCapture("AiTrainer/curls.mp4")
detector = pm.poseDetector()
count = 0
dir = 0
pTime = 0
while True:
    success, img = cap.read()
    img = cv2.resize(img, (1280, 720))
    # img = cv2.imread("AiTrainer/test.jpg")
    img = detector.findPose(img, False)
    lmList = detector.findPosition(img, False)
    # print(lmList)
    if len(lmList) != 0:
        # Right Arm
        angle = detector.findAngle(img, 12, 14, 16)
        # # Left Arm
        #angle = detector.findAngle(img, 11, 13, 15,False)
        per = np.interp(angle, (210, 310), (0, 100))
        bar = np.interp(angle, (220, 310), (650, 100))
        # print(angle, per)
        # Check for the dumbbell curls
        color = (255, 0, 255)
        if per == 100:
            color = (0, 255, 0)
            if dir == 0:
                count += 0.5
                dir = 1
        if per == 0:
            color = (0, 255, 0)
            if dir == 1:
                count += 0.5
                dir = 0
        print(count)
        # Draw Bar
        cv2.rectangle(img, (1100, 100), (1175, 650), color, 3)
        cv2.rectangle(img, (1100, int(bar)), (1175, 650), color, cv2.FILLED)
        cv2.putText(img, f'{int(per)} %', (1100, 75), cv2.FONT_HERSHEY_PLAIN, 4,
                    color, 4)
        # Draw Curl Count
        cv2.rectangle(img, (0, 450), (250, 720), (0, 255, 0), cv2.FILLED)
        cv2.putText(img, str(int(count)), (45, 670), cv2.FONT_HERSHEY_PLAIN, 15,
                    (255, 0, 0), 25)
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    cv2.putText(img, str(int(fps)), (50, 100), cv2.FONT_HERSHEY_PLAIN, 5,
                (255, 0, 0), 5)
    cv2.imshow("Image", img)
    cv2.waitKey(1)

# AI Virtual Mouse Project(Not Completed)

In [6]:
# !pip install autopy

In [7]:
import cv2
import numpy as np
import HandTrackingModule as htm
import time
import autopy

In [17]:
wCam, hCam = 640, 480
frameR = 100 # Frame Reduction
smoothening = 7
#########################

pTime = 0
plocX, plocY = 0, 0
clocX, clocY = 0, 0

cap = cv2.VideoCapture(1)
cap.set(3, wCam)
cap.set(4, hCam)
detector = htm.handDetector(maxHands=1)
wScr, hScr = autopy.screen.size()
# print(wScr, hScr)

while True:
# 1. Find hand Landmarks
    success, img = cap.read()
    img = detector.findHands(img)
    lmList, bbox = detector.findPosition(img)
    # 2. Get the tip of the index and middle fingers
    if len(lmList) != 0:
        x1, y1 = lmList[8][1:]
        x2, y2 = lmList[12][1:]
    # print(x1, y1, x2, y2)

    # 3. Check which fingers are up
    fingers = detector.fingersUp()
    # print(fingers)
    cv2.rectangle(img, (frameR, frameR), (wCam-frameR, hCam-frameR),
    (255, 0, 255), 2)
    # 4. Only Index Finger : Moving Mode
    if fingers[1] == 1 and fingers[2] == 0:
    # 5. Convert Coordinates
        x3 = np.interp(x1, (frameR, wCam - frameR), (0, wScr))
        y3 = np.interp(y1, (frameR, hCam - frameR), (0, hScr))
    # 6. Smoothen Values
    clocX = plocX + (x3 - plocX) / smoothening
    clocY = plocY + (y3 - plocY) / smoothening

    # 7. Move Mouse
    autopy.mouse.move(wScr - clocX, clocY)
    cv2.circle(img, (x1, y1), 15, (255, 0, 255), cv2.FILLED)
    plocX, plocY = clocX, clocY

    # 8. Both Index and middle fingers are up : Clicking Mode
    if fingers[1] == 1 and fingers[2] == 1:
    # 9. Find distance between fingers
        length, img, lineInfo = detector.findDistance(8, 12, img)
        print(length)
    # 10. Click mouse if distance short
    if length < 40:
        cv2.circle(img, (lineInfo[4], lineInfo[5]),
        15, (0, 255, 0), cv2.FILLED)
        autopy.mouse.click()

    # 11. Frame Rate
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    cv2.putText(img, str(int(fps)), (20, 50), cv2.FONT_HERSHEY_PLAIN, 3,
    (255, 0, 0), 3)
    # 12. Display
    cv2.imshow('Image', img)
    if cv2.waitKey(1) & 0xff == ord('q'):   
        break

ValueError: not enough values to unpack (expected 2, got 0)